In [1]:

library(parallel)
library(doParallel)
library(future)
library(iterators)
library(listenv)

options(future.globals.maxSize = 8 * 1024^3) # increase dataset size limit taken by future to 8GB

# detects number of cores
n.cores <- parallel::detectCores()


plan(list(
  tweak(multisession, workers = ((n.cores + 5) %/% 3) %/% 2),
  tweak(multisession, workers = ((n.cores + 5) %/% 3) %/% 2),
  tweak(multisession, workers = 3)
))


Lade nötiges Paket: foreach

Lade nötiges Paket: iterators



In [2]:
start.time <- Sys.time()

In [3]:
# ---------- Script ----------
# input directory
input_dir <- getwd()
#input_dir
#input_dir <- "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/SmarinoiRun1"

In [4]:
# load the functions file
source(paste(getwd(),"/Workflow_R_Functions.r", sep = ""))
#source("/Users/mahnoorzulfiqar/OneDriveUNI/MAW/R/Workflow_R_Functions.r")

Lade nötiges Paket: S4Vectors

Lade nötiges Paket: stats4

Lade nötiges Paket: BiocGenerics


Attache Paket: ‘BiocGenerics’


Die folgenden Objekte sind maskiert von ‘package:stats’:

    IQR, mad, sd, var, xtabs


Die folgenden Objekte sind maskiert von ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min



Attache Paket: ‘S4Vectors’


Das folgende Objekt ist maskiert ‘package:future’:

    values


Die folgenden Objekte sind maskiert von ‘package:base’:

    I, expand.grid, unname


Lade nötiges Paket: BiocParallel

Lade nötiges Paket: ProtGenerics


Attache Paket: ‘ProtGenerics’


Das folgende Objekt ist maskiert ‘package:stats’:

 

In [5]:
input_table <- data.frame(ms2_rfilename(input_dir))
input_table

mzml_files,ResultFileNames,File_id
<chr>,<chr>,<chr>
./DS200309_Scost_QC_70k_neg_PRM.mzML,./DS200309_Scost_QC_70k_neg_PRM,file_1
./DS200309_Scost_QC_70k_pos_PRM.mzML,./DS200309_Scost_QC_70k_pos_PRM,file_2
./DS_201124_SC_full_PRM_neg_01.mzML,./DS_201124_SC_full_PRM_neg_01,file_3
./DS_201124_SC_full_PRM_neg_02.mzML,./DS_201124_SC_full_PRM_neg_02,file_4
./DS_201124_SC_full_PRM_neg_03.mzML,./DS_201124_SC_full_PRM_neg_03,file_5
./DS_201124_SC_full_PRM_neg_04.mzML,./DS_201124_SC_full_PRM_neg_04,file_6
./DS_201124_SC_full_PRM_neg_05.mzML,./DS_201124_SC_full_PRM_neg_05,file_7
./DS_201124_SC_full_PRM_neg_06.mzML,./DS_201124_SC_full_PRM_neg_06,file_8
./DS_201124_SC_full_PRM_neg_07.mzML,./DS_201124_SC_full_PRM_neg_07,file_9


In [6]:
input_table_idxs <- listenv()

In [ ]:
for (i in 1:nrow(input_table)){
    input_table_idxs[[i]] <- future({
    #Preprocess and Read the mzMLfiles
    spec_pr <- spec_Processing(input_dir,
                               input_table[i, "mzml_files"],
                               input_table[i, "ResultFileNames"])

    # Extract MS2 peak lists
    spec_pr2 <- ms2_peaks(pre_tbl = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/premz_list.txt", sep = ""), "."), sep =""),
                          proc_mzml = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/processedSpectra.mzML", sep = ""), "."), sep =""),
                          input_dir,
                          result_dir = input_table[i, "ResultFileNames"],
                         file_id = input_table[i, "File_id"])
        
    # camera results for isotopes
    cam_res <- cam_func(input_dir,
                        f = input_table[i, "mzml_files"], 
                        ms2features = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/insilico/MS2DATA.csv", sep = ""), "."), sep = ""))
    

    # Extract MS1 peaks or isotopic peaks
    ms1p <- ms1_peaks(x = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/insilico/MS2DATA.csv', sep = ""), "."), sep =""),
                      y = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/CAMERAResults.csv', sep = ""), "."), sep =""), 
                      input_table[i, "ResultFileNames"],
                      input_dir,
                      QCfile = TRUE)

    #prepare sirius parameter files
    sirius_param_files <- sirius_param(x = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/insilico/MS1DATA.csv', sep = ""), "."), sep =""),
                                       result_dir = input_table[i, 'ResultFileNames'],
                                       input_dir,
                                       SL = TRUE)


    # Run sirius
    run_sirius(files = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/insilico/MS1DATA_SiriusPandSL.tsv', sep = ""), "."), sep =""),
               ppm_max = 5,
               ppm_max_ms2 = 15,
               QC = TRUE,
               SL = TRUE,
               SL_path = paste(getwd(), "/SL_Frag/", sep = ""),
               candidates = 30,
              profile = "orbitrap")

   }) #end input_table_idxs future
}


In [ ]:
input_table_idxs <- as.list(input_table_idxs)
v_input_table_idxs <- future::value(input_table_idxs)

In [ ]:
end.time <- Sys.time()

In [ ]:
time.taken <- end.time - start.time
print(time.taken)